In [17]:
## for data
import numpy as np
import pandas as pd
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for geospatial
import folium
import geopy
## for machine learning
from sklearn import preprocessing, cluster
import scipy
## for deep learning


import minisom

In [26]:
listings = pd.read_csv('./listings.csv')
d_listings = pd.read_csv("listings_summary.csv")

calendar = pd.read_csv("./calendar.csv")
reviews = pd.read_csv("./reviews.csv")
d_reviews = pd.read_csv("./reviews_summary.csv")
neighbourhoods = pd.read_csv("./neighbourhoods.csv")

C:\Users\Abhishek\AppData\Local\Temp\ipykernel_20684\2256974550.py:1: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  listings = pd.read_csv('./listings.csv')


In [27]:
listings.head()


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,11551,https://www.airbnb.com/rooms/11551,20191105115249,2019-11-06,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,"Amenities Bedding: 1 Double bed, 1 living room...",Unlike most rental apartments out there my fla...,family,Not even 10 minutes by metro from Victoria Sta...,...,t,f,strict_14_with_grace_period,f,t,2,2,0,0,1.58
1,38151,https://www.airbnb.com/rooms/38151,20191105115249,2019-11-06,Double room/ lounge,NaN,"Comfortable, large double room /lounge area av...","Comfortable, large double room /lounge area av...",none,NaN,...,f,f,flexible,f,f,1,0,1,0,NaN
2,13913,https://www.airbnb.com/rooms/13913,20191105115249,2019-11-06,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,...,f,f,moderate,f,f,2,1,1,0,0.17
3,38407,https://www.airbnb.com/rooms/38407,20191105115249,2019-11-06,Canary Wharf Immaculate Apt for 2,"The bright, light and stylish apartment in Can...","An entire bright, light and stylish apartment....","The bright, light and stylish apartment in Can...",none,Very easy to get to all the main sites. The tu...,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,1.23
4,90700,https://www.airbnb.com/rooms/90700,20191105115249,2019-11-06,Sunny Notting Hill flat & terrace,This is a home not a hotel - for the cost gues...,This charming 1 bedroom with en-suite bathroom...,This is a home not a hotel - for the cost gues...,none,A quick guide or a run through about the area ...,...,t,f,strict_14_with_grace_period,f,f,2,2,0,0,3.33


In [28]:
d_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,11551,Arty and Bright London Apartment in Zone 2,43039,Adriano,NaN,Lambeth,51.46225,-0.11732,Entire home/apt,88,3,185,2019-09-15,1.58,2,336
1,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56802,-0.11121,Private room,65,1,19,2019-10-07,0.17,2,365
2,90700,Sunny Notting Hill flat & terrace,491286,Chil,NaN,Kensington and Chelsea,51.51074,-0.19853,Entire home/apt,105,2,339,2019-07-30,3.33,2,268
3,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.48796,-0.16898,Entire home/apt,100,30,88,2019-09-23,0.73,1,158
4,92399,"MODERN SELF CONTAINED ARCHITECT FLATLET, ISLIN...",497366,Andrea & Mark,NaN,Islington,51.55071,-0.08547,Private room,77,1,207,2019-10-21,2.04,2,336


In [29]:
import geopandas as gpd
neighbourhoods = gpd.read_file('./neighbourhoods.geojson')

In [30]:
nhood_map_sf = neighbourhoods.to_crs(epsg=4326)


In [31]:
listings[['id', 'host_id']] = listings[['id', 'host_id']].astype(str)
reviews[['listing_id']] = reviews[['listing_id']].astype(str)
d_reviews[['id', 'reviewer_id', 'listing_id']] = d_reviews[['id', 'reviewer_id', 'listing_id']].astype(str)
calendar[['listing_id']] = calendar[['listing_id']].astype(str)
d_listings[['host_id', 'id']] = d_listings[['host_id', 'id']].astype(str)

# Change price in detailed listings to numerical
# Remove $ and , symbol in columns where currency is read as character.
strip_dollars = lambda x: pd.to_numeric(x.replace('$', '').replace(',', ''))
d_listings.iloc[:, 61:65] = d_listings.iloc[:, 61:65].applymap(strip_dollars)
d_listings.iloc[:, 67] = d_listings.iloc[:, 67].apply(strip_dollars)

KeyError: "['id', 'reviewer_id'] not in index"